<a href="https://colab.research.google.com/github/HrisheekAsanabada/CMPE257-Fall23-Project-Optiver-Trading-at-the-Close/blob/017422717/ML_Project_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.7/981.7 kB 8.8 MB/s eta 0:00:00


Implementing ARIMA model for a single stock

In [2]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import joblib
from statsmodels.tsa.arima.model import ARIMA as ARIMA
from arch import arch_model
from sklearn.metrics import r2_score
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.options.mode.chained_assignment = None
PATH_DATASET = "/content/drive/MyDrive/MLProject/optiver-trading-at-the-close"

def gts_cv(date_ids, holdout_size):

    X = date_ids[['date_id']]
    groups = X['date_id'].values
    unique_date_ids = np.unique(groups)
    array_split_date_ids = np.array_split(unique_date_ids, len(unique_date_ids) // holdout_size)[::-1]
    array_split_date_ids = np.flip(array_split_date_ids)

    for date_ids in array_split_date_ids:


        test_condition = X['date_id'].isin(np.array([date_ids[-1]]))
        val_index = X.loc[test_condition].index
        train_condition = X['date_id'].isin(date_ids)
        train_index = X.loc[train_condition].index
        train_index = pd.Index(set(train_index)- set(val_index))


        yield train_index, val_index


def mae(y_true, y_pred):
    return np.abs(y_true - y_pred).mean()


def feval_mae(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'MAE', mae(y_true, y_pred), False

def mse(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()



def train_and_evaluate_ARIMA(train):

    train = train[train['stock_id'] == 0]
    date_ids_train = train[['date_id','target']].reset_index(drop=True)

    cvs_dict = {'Fold': [],  'MAE_val': [],'MSE_val':[]}
    cv = gts_cv(date_ids_train, holdout_size=7)
    fold = 1
    for train_index, val_index in cv:

        target_train_cv, target_val_cv = date_ids_train.iloc[train_index], date_ids_train.iloc[val_index]


        print("Training set date range: {} -- {}".format(target_train_cv.date_id.min(),target_train_cv.date_id.max()))
        print("Validation set date range: {} -- {}".format(target_val_cv.date_id.min(),target_val_cv.date_id.max()))

        target_train_cv =target_train_cv.drop(['date_id'], axis = 1)
        target_val_cv = target_val_cv.drop(['date_id'], axis = 1)


        model = ARIMA(target_train_cv, order=(2, 1, 6))
        fitted = model.fit()
        fc = fitted.forecast(len(target_val_cv))
        score=mae(fc.values,target_val_cv.values)
        mse_score=mse(fc.values,target_val_cv.values)
        print(fc.values,score)
        cvs_dict['Fold'].append(fold)
        cvs_dict['MAE_val'].append(score)
        cvs_dict['MSE_val'].append(mse_score)
        fold += 1
        true_values = target_val_cv.values
        mean_true_values = np.mean(true_values)
        variance_true_values = np.mean((true_values - mean_true_values) ** 2)


    print(f'CV Val MAE:',np.asarray(cvs_dict['MAE_val']).mean())
    print(f'CV Val MSE:',np.asarray(cvs_dict['MSE_val']).mean())
    return model


dtypes = {
    'stock_id' : np.uint8,
    'date_id' : np.uint16,
    'seconds_in_bucket' : np.uint16,
    'imbalance_buy_sell_flag' : np.int8,
    'time_id' : np.uint16
    }

# df =  pd.read_csv('/content/drive/MyDrive/MLProject/data_after_preprocessing (4).csv')
df =  pd.read_csv(os. path.join(PATH_DATASET, "train.csv"))
model=train_and_evaluate_ARIMA(df)


Training set date range: 0 -- 6
Validation set date range: 7 -- 7


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[-4.35310482 -3.2706266  -2.67613122 -2.06256964 -2.08271601 -0.80803274
 -0.9202005  -0.82792022 -0.84320803 -0.83590405 -0.83752905 -0.83691485
 -0.83707348 -0.83701992 -0.83703484 -0.83703008 -0.83703146 -0.83703103
 -0.83703116 -0.83703112 -0.83703113 -0.83703113 -0.83703113 -0.83703113
 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113
 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113
 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113
 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113
 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113 -0.83703113
 -0.83703113] 2.9014688425401753
Training set date range: 8 -- 14
Validation set date range: 15 -- 15


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be i

[ 1.75394863  0.5766537   1.11818798  1.27320603 -0.22706863 -0.97220319
 -0.42782241 -0.39599494 -0.531298   -0.50342787 -0.47913277 -0.49197233
 -0.49429088 -0.49069068 -0.49108893 -0.49182494 -0.49154004 -0.49144265
 -0.49153452 -0.49153327 -0.49151217 -0.49151796 -0.49152138 -0.49151913
 -0.49151892 -0.4915195  -0.4915194  -0.49151929 -0.49151934 -0.49151935
 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934
 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934
 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934
 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934 -0.49151934
 -0.49151934] 2.0856223593096015
Training set date range: 16 -- 22
Validation set date range: 23 -- 23


...

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


[3.94363947 2.25513159 2.68147173 0.87025787 0.58497838 0.67049675
 0.67766439 0.66977539 0.67059982 0.6710676  0.67092178 0.670911
 0.6709242  0.67092273 0.67092196 0.67092221 0.67092223 0.6709222
 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221
 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221
 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221
 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221
 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221
 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221 0.67092221
 0.67092221] 3.291719350862301
Training set date range: 474 -- 479
Validation set date range: 480 -- 480


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[ 3.78525568  3.26008466  3.1587133   0.99630783 -0.02270353 -1.49794763
 -0.94763551 -0.80891013 -0.94492711 -0.93733001 -0.91397168 -0.92110212
 -0.92368639 -0.92173127 -0.92173104 -0.9220936  -0.92200483 -0.92195935
 -0.92198695 -0.92198862 -0.9219831  -0.92198414 -0.92198491 -0.92198453
 -0.92198448 -0.92198456 -0.92198455 -0.92198454 -0.92198454 -0.92198454
 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454
 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454
 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454
 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454 -0.92198454
 -0.92198454] 3.2754535624398278
CV Val MAE: 4.522616776568709
CV Val MSE: 36.72773816005741


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [3]:
import pickle

# Assuming 'model' is your trained model object (e.g., ARIMA or GARCH)
# Save the model to a file
with open('arima_model.pkl', 'wb') as file:
    pickle.dump(model, file)